# Table of Contents
[1. Model Preparation](#1.-Model-Preperation)
<br>
* [1.1 Reviewing, Splitting data set](#1.1-Reviewing,-splitting-dataset-into-7:3-for-training-and-testing.)
* [1.2 Plotting features against target feature](#1.2-Plot-to-compare-all-features-to-target-feature-to-help-make-decisions-to-keep-for-the-models.)
    * [1.2.1 Plotting datetime feature against target feature](#Plotting-datetime-feature-against-target-feature)
    * [1.2.2 Plotting numerical features against target feature](#Plotting-numerical-features-against-target-feature)
    * [1.2.3 Plotting categorical features against target feature](#Plotting-categorical-features-against-target-feature)
* [1.3. Summary of all features](#1.3.-Summary-of-all-features)
    * [1.3.1 Numerical Features](#Numerical-Features)
    * [1.3.1 Cateogrical Features](#Categorical-Features)
* [2. Linear Regression](#2.-Linear-Regression)
* [3. Route model and taking the proportion of the prediction to calculate a journey time for the user](#3.-Route-model-and-taking-the-proportion-of-the-prediction-to-calculate-a-journey-time-for-the-user.)
    * [3.1 Calculating the proportion of each stop from the overall trip](#3.1-Calculating-the-proportion-of-each-stop-from-the-overall-trip.)
* [4. Stop pair model](#4.-Stop-pair-model)

Establishing a connection with sqlite database

In [ ]:
# import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sqlite3
import pickle

# from sagemaker import get_execution_role
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from math import log
from statistics import stdev
from statistics import mode


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Connecting to s3
# role = get_execution_role()
# bucket='sagemaker-studio-520298385440-7in8n1t299'
# data_key = 'route_46a.feather'
# data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except 'Error' as e:
        print(e)
        
    return conn

sumo = """SELECT COUNT(*) from leavetimes"""
s = conn.execute(sumo)
for x in s:
    print(x)

In [ ]:
# create connection to db
db_file = "C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/dublinbus.db"
conn = create_connection(db_file)

In [ ]:
# initialise query
query = """
SELECT leavetimes.*, weather.*
FROM leavetimes, weather
WHERE TRIPID in  
    (SELECT TRIPID
    FROM trips
    WHERE LINEID = '46A' AND DIRECTION = '1')
AND leavetimes.DAYOFSERVICE = weather.dt;
"""

In [ ]:
# execute query and read into dataframe
query_df = pd.read_sql(query, conn)

# 1. Model Preperation

In [ ]:
# Loading file
df = query_df

## 1.1 Reviewing, splitting dataset into 7:3 for training and testing.

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
# Missing values
df.isnull().sum()

In [ ]:
# Unique types for each feature
df.nunique()

In [ ]:
# Datatypes and convert
df.dtypes

In [ ]:
# Rows and columns
df.shape

**Review so far:**
<br>
There are no more missing values and the constant columns have been removed.
* Remove index, index, dt.
* Investigate level_0.
* Convert the following to categorical: DAYOFWEEK, MONTHOFSERVICE, PROGRNUMBER, STOPPOINTID, VEHICLEID, IS_HOLIDAY, IS_WEEKDAY, TRIPID, weather_id, weather_main, weather_description
* We have data for most of the days of the year and for each month.


In [ ]:
df = df.drop(['index', 'level_0', 'dt'], axis=1)

In [ ]:
# Sorting by trip then dayofservice
df = df.sort_values(by=['TRIPID', 'DAYOFSERVICE', 'PROGRNUMBER'])

In [ ]:
# Creating features
categorical_features = ['DAYOFWEEK', 'MONTHOFSERVICE', 'PROGRNUMBER', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID',
                       'IS_HOLIDAY', 'IS_WEEKDAY', 'TRIPID', 'VEHICLEID', 'weather_id', 'weather_main', 'weather_description']

datetime_features = ['DAYOFSERVICE']

numerical_features = ['PLANNEDTIME_ARR', 'ACTUALTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_DEP',
                     'DWELLTIME', 'PLANNEDTIME_TRAVEL', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'clouds_all']

target_feat = 'ACTUALTIME_TRAVEL'

In [ ]:
# Converting object to categorical
for column in categorical_features:
    df[column] = df[column].astype('category')
    
# Converting dayofservice to datetime
df['DAYOFSERVICE'] = pd.to_datetime(df['DAYOFSERVICE'])

In [ ]:
# Replacing PROGRNUMBER equal to 1 of ACTUALTIME_TRAVEL with 0
df.loc[df['PROGRNUMBER'] == '1', 'ACTUALTIME_TRAVEL'] = 0
df.loc[df['PROGRNUMBER'] == '1', 'PLANNEDTIME_TRAVEL'] = 0

In [ ]:
df.loc[df['PLANNEDTIME_TRAVEL'] < 0, 'PLANNEDTIME_TRAVEL'] = 0
df.loc[df['ACTUALTIME_TRAVEL'] < 0, 'ACTUALTIME_TRAVEL'] = 0

In [ ]:
# Making new feature for previous stoppointid and let those with PROGRNUMBER = 1 to 0
# df['PREVIOUS_STOPPOINTID'] = df['STOPPOINTID'].shift()
# first_stop = {'0':'0'}
# df['PREVIOUS_STOPPOINTID'] = df['PREVIOUS_STOPPOINTID'].cat.add_categories(first_stop)
# df.loc[df['PROGRNUMBER'] == '1', 'PREVIOUS_STOPPOINTID'] = '0'

<br><br>
Setting the target feature as _y and x_ as the remaining features in the dataframe. 
<br><br>

In [ ]:
df.set_index(np.random.permutation(df.index))
# sort the resulting random index
df.sort_index(inplace=True)
df.head(5)

In [ ]:
# Creating y and x axis
target_feature = df['ACTUALTIME_TRAVEL']
y = pd.DataFrame(target_feature)
X = df.drop(['ACTUALTIME_TRAVEL'], axis=1)

# Splitting dataset for train and testing data by 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Printing shape of the new split data
print("The original range is: ",df.shape[0])
print("The training range (70%):\t rows 0 to", round(X_train.shape[0]))
print("The test range (30%): \t rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

In [ ]:
X_train.head(5)

## 1.2 Plot to compare all features to target feature to help make decisions to keep for the models.

#### Plotting datetime feature against target feature

In [ ]:
# Plot datetime feature against target feature
X_train.DAYOFSERVICE = pd.to_numeric(X_train.DAYOFSERVICE)
df_temp = pd.concat([X_train['DAYOFSERVICE'], y_train], axis=1)
correlation_dt = df_temp[['DAYOFSERVICE', 'ACTUALTIME_TRAVEL']].corr(method='pearson')
correlation_dt

In [ ]:
fig = plt.figure()
ax = fig.add_subplot
df_temp.plot(kind='scatter', x='DAYOFSERVICE', y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', 'DAYOFSERVICE']].corr().to_numpy()[0,1], figsize=(15, 8)) 

#### Plotting numerical features against target feature

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    correlation_dt = df_temp[[column, 'ACTUALTIME_TRAVEL']].corr(method='pearson')
    print('\n',correlation_dt)

Using pearson correlation, we see that the correlation between 

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    correlation_dt = df_temp[[column, 'ACTUALTIME_TRAVEL']].corr(method='spearman')
    print('\n',correlation_dt)

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='pearson').to_numpy()[0,1], figsize=(12, 8)) 

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='spearman').to_numpy()[0,1], figsize=(12, 8)) 

In [ ]:
df_numeric = df[numerical_features]
for feature in df_numeric:
    df_numeric[feature] = np.log(df_numeric[feature])
df_numeric['ACTUALTIME_TRAVEL'] = np.log(df['ACTUALTIME_TRAVEL'])

In [ ]:
# Creating y and x axis
target_feature_numeric = df_numeric['ACTUALTIME_TRAVEL']
y_numeric = pd.DataFrame(target_feature_numeric)
X_numeric = df_numeric.drop(['ACTUALTIME_TRAVEL'], axis=1)

# Splitting dataset for train and testing data by 70/30
X_train_numeric, X_test_numeric, y_train_numeric, y_test_numeric = train_test_split(X_numeric, y_numeric, test_size=0.3, random_state=1)

# Printing shape of the new split data
print("The original range is: ",df.shape[0])
print("The training range (70%):\t rows 0 to", round(X_train_numeric.shape[0]))
print("The test range (30%): \t rows", round(X_train_numeric.shape[0]), "to", round(X_train_numeric.shape[0]) + X_test_numeric.shape[0])

for column in numerical_features:
    df_temp = pd.concat([X_train_numeric[column], y_train_numeric], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='spearman').to_numpy()[0,1], figsize=(12, 8)) 

#### Plotting categorical features against target feature

In [ ]:
year_features = ['DAYOFWEEK', 'IS_HOLIDAY', 'IS_WEEKDAY', 'MONTHOFSERVICE', 'weather_id', 'weather_main', 'weather_description']

for feature in year_features:
    print(feature)
    df_temp = pd.concat([X_train, y_train], axis=1)
    unique = df_temp[feature].unique()
    list_average = []
    
    for value in unique:
        list_values = df_temp[df_temp[feature]== value]['ACTUALTIME_TRAVEL'].tolist()
        length_list = len(list_values)
        average =  sum(list_values)/length_list
        list_average += [average]
#         print(f'Sum of values / list of values: \n {sum(list_values)} / {length_list}')
#         print(f'Average ACTUALTIME_TRAVEL: {average}, \n')
        
    # taken from https://pythonspot.com/matplotlib-bar-chart/
    y_pos = np.arange(len(unique))
    plt.bar(y_pos, list_average, align='center')
    plt.xticks(y_pos, unique)
    plt.ylabel('Usage')
    plt.title(feature)
    plt.xticks(rotation=90)

    plt.show()

I think there is some outliers in ACTUALTME_TRAVEL. The averages are all in negatives which suggests that the travel times. Would that be an outlier if the negative values are very great? 
<br><br>
**DAYOFWEEK:**
The lowest average is Sunday and the busiest is Monday. So it does make a difference.
<br><br>
**IS_WEEKDAY:**
The same comment that there is a difference in average times.
<br><br>
**MONTHOFSERVICE:**
Interestingly enough, they have difference averages depending on each month with August being the least busiest and April being the busiest. It must have something to do with the weather maybe?


In [ ]:
# Average time for each vehicle id
df_temp = pd.concat([X_train, y_train], axis=1)
vehicleid = df_temp['VEHICLEID'].unique().tolist()
for id_ in vehicleid:
    print(f'VEHICLEID: {id_}')
    list_values = df_temp[df_temp['VEHICLEID']== id_]['ACTUALTIME_TRAVEL'].tolist()
    length_list = len(list_values)
    average =  sum(list_values)/length_list
    print(f'Average ACTUALTIME_TRAVEL: {average} \n')

In [ ]:
# Making dummy variables for categorical 
cat = ['DAYOFWEEK', 'MONTHOFSERVICE', 'PROGRNUMBER', 'STOPPOINTID', 'IS_HOLIDAY', 'IS_WEEKDAY', 'weather_id', 'weather_main', 'weather_description']
df_temp = pd.concat([X_train, y_train], axis=1)
df_copy = df_temp.copy()
df_copy = df_copy[cat]
df_copy = pd.get_dummies(df_copy)
df_copy = pd.concat([df_copy, y_train], axis=1)

categorical_corr = df_copy.corr()['ACTUALTIME_TRAVEL'][:]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(categorical_corr)

In [ ]:
categorical_list = categorical_corr[categorical_corr > 0.04].index.tolist()
categorical_list.remove('ACTUALTIME_TRAVEL')

In [ ]:
categorical_list

## 1.3. Summary of all features
<br><br>
#### Numerical Features
<br><br>

**DayOfService:**
* The correlation to the target feature is very low of 0.03806.
* Don't see it being a useful feature for the target feature. 
* Plot represents a straight line, which suggests little to no correlation.
* Conclusion: dropped because of the low correlation score. 

**PlannedTime_Arr:**
* There is very low correlation against the target feature though it gets better using spearman correlation.
* After logging the data, the correlation plot did not make a huge difference when using the spearman method to plot it for the second time. 
* Pearson and spearman plot pre log suggests little correlation as it is a continuous straight line. However, this shouldn't mean it should be dropped.
* When most values in the target feature fell less than 10, we see that the plannedtime arrival values increasing, it didn't change much. This would be due to the fact that the target feature is the difference between times so it would make sense that the relationship is poor.
* After logging the data, the plot is more spread out instead of a straight line, but the correlation score still shows a similar low score with a .02 difference using the spearman method. 
* Conclusion: However, this will be dropped.

**ActualTime_Arr:**
* Compared to Planned time arrival feature, the pearson correlation score is poorer but the spearman scores are more similar pre log. 
* It is similar to planned time arrival in that the plot represents a straight line, that suggests a poor relationship with the target feature. 
* After logging the data, it is found that the plot is more spread out. The score using spearman is not much different pre logging the data. 
* However, it would be unwise to drop this feature as it I feel it would serve good purpose for the target feature for predicting the prediction time for the next stop. 
* Conclusion: this will be dropped.

**PlannedTime_Dep:**
* Planned time departure has little correlation with the target feature after looking at spearman and pearsons. 
* It doesn't have a linear relationship and the straight line on the plot of both methods proves this.
* However, when plotted using the logged values we see that the correlation score hasn't changed but the data is more spread out. 
* This doesn't change the relationship much, however. 
* Even so, this will be kept as I feel it would help the predictions. Having the planned time departures would help skew a better result because it would relatively be close to the actual time departure even though it is just an estimate.
* Conclusion: this will be dropped 

**ActualTime_Dep:**
* Actual time departure is again, more or less the same. It represents the departure for these times at a particular stop to go to the next stop. It is strange that the correlation is so low even after logging the data but it would make sense as you wouldn't expect there to be a linear relationship.
* The plot is similar to the rest of the previous features mentioned so far. 
* However, it will still be kept because I feel it would still be a useful feature for predicting a time in seconds. 
* By taking the actual time departure for a particular stop it may help.
* Conclusion: this will be dropped.

**Dwell Time:**
* Dwell time has a 0.03 coorelation score with the target feature. It suggests on the graph that the time for dwell time equal to 0 then the more the target feature time increases. It might suggest traffic times where if a bus is full then it might be due to rush hour? busy hours?
* Plotting against the target feature after logging the data gives similar scores using the spearman correlation method. However we see the graph differing from pre log plot. It is more grouped up together compared to the previous graph plot.
* Because the score is more fairer compared to the previous, it will be useful to keep it for the modelling.
* Conclusion: dropped.

**PlannedTime_Travel:**
* When plotting using the pearse correlation method, it gave a correlation of 0.2. This time it is the highest correlation and we see a small linear relationship.
* The time for planned time travel, as it increases, so does the target feature. It gives us an indication of that slight linear relationship.
* Using spearmans to graph the correlation gave us a 0.7 score which is a good indication that the two features has a linear relationship.
* Because of this, this feature will be dropped.

**Temp:**
* Temp  has a negative 0.009 correlation with the target feature and an even poorer linear relationship at -.002.
* This indicates a poor linear/monotonic relationship and it will not serve useful for the model.
* The graph plots does not give anymore useful information that would give further evidence that it should be kept.
* Conclusion: drop.

**Pressure:**
* It also has a negative linear relationship with the target feature.
* When looking at the graph plots for both spearman and pearsons, it does not give any further insights.
* For this reason, this feature will be dropped.

**Humidity:**
* Humidity does not have a strong relationship with the target feature, be it linear or monotonic.
* The reason being the correlation using both methods fell < 0.00. 
* Unfortunately, the graph does not represent anything useful either.
* When looking at the logged data plots however, there is a slight difference however it is not signficant enough that this feature should still be kept as there is no distinct relationship that we can see.
* Conclusion: drop.

**Windspeed:**
* No linear relationship.
* Indicates a small monotonic relationship.
* This means that as the windspeed value increases, the value of the target feature tends to be higher as well.
* But a spearman correlation of 0.01 is not strong enough of a feature to keep.
* Conclusion: drop

**Wind_Deg:**
* This feature will be dropped immediately as the correalations are both <0.000.

**Rain_1H:**
* It doesn't have a strong linear relationship but it shows spearmans correlation some promising results when the data has been logged.


<br><br>
#### Categorical Features
<br><br>
**DayOfWeek:**
* In the graph we see the actual time travel increasing during weekdays and slowly the travel time is less during weekends. 
* This suggests a relationship between the days of the week and the target feature in which weekdays have a higher tendency for the actualtime travel feature to be higher.
* Conclusion: this will be kept.

**MonthofService:**
* In the graph, we don't really see a connection between each month against the target feature even if it is in order. 
* The overall actual travel time is higher in february before it dips, then rising during winter season.
* The correlation score seems to be poor also for each month. 
* This feature will still be kept. 

**Progrnumber:**
* Most progrnumbers will be dropped as a lot of the correlations are <0.00.
* For this reason, this feature will be dropped.
    
**StoppointID:**
* Similarly to progrnumbers, there are a lot of low correlations falling <0.00.
* Most stoppoint numbers are <0.00 correlation.
* This indicates a very low relationship with the target feature. 
* For this reason, this feature will be dropped, except for those with a correlation > 0.04
    
**Is_Holiday:**
* After analyzing the graph, we see a relationship between the target feature and whether or not the time falls under a holiday date (non-school holiday).
* If it a non holiday, the actual time travel increases. 
* If it is a holiday, the actual time travel decreases. 
* This means that less people are using public transport if it is a holiday date.
* For this reason, this feature will be kept.

**Is_Weekday:**
* Like Is_Holiday, we see a relationship between the target feature and whether or not the time is during a weekday or not. 
* We see a contrast between the two values in which 1, being a weekday, has a higher actual time travel, vice versa.
* For this reason, it is a good indication of a relationship to the target feature.
* Therefore, this feature will be kept. 

**VehicleID:**
* When looking at the different averages, we see that the average differences are not big.
* For this reason, it may be best to drop this feature because it doesn't give any indication it would be a useful feature to help the prediction models.


## 1.4 Cleaning up features

### Setting low correlation features - keep

In [ ]:
# Categorical features
low_corr_categorical = ['DAYOFWEEK', 'MONTHOFSERVICE', 'IS_HOLIDAY', 'IS_WEEKDAY'] 

### Setting low correlation features - drop

In [ ]:
# Numerical features
low_corr_numerical = ['PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP','PLANNEDTIME_TRAVEL']

low_corr = ['DAYOFSERVICE', 'VEHICLEID', 'TRIPID', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID', 'PROGRNUMBER', 'temp', 'pressure', 'humidity', 
            'wind_speed', 'wind_deg', 'weather_id', 'weather_description', 'clouds_all', 'PREVIOUS_STOPPOINTID', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP',
           'PLANNEDTIME_TRAVEL', 'DWELLTIME']

### Setting high correlation  features

In [ ]:
# Numerical features 
high_corr_numerical = ['DWELLTIME', 'PLANNEDTIME_TRAVEL']

### Dropping features & setting dummy features

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(low_corr, 1)
df_copy

In [ ]:
df_copy = pd.get_dummies(df_copy)
df_copy

### Training & Testing data

In [ ]:
# All features
features = df_copy.columns.tolist()
features

In [ ]:
datas = {'ACTUALTIME_TRAVEL': df_copy['ACTUALTIME_TRAVEL']}
y = pd.DataFrame(data=datas)
X = df_copy.drop(['ACTUALTIME_TRAVEL'],1)

In [ ]:
# Splitting the dataset into 2 datasets: 
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

print("The Original range of the dataset: ",df.shape[0])
print("The Training range taken from dataset: (70%): rows 0 to", round(X_train.shape[0]))
print("The Test range taken from dataset: (30%): rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

In [ ]:
print("\nDescriptive features in X:\n", X_train.head(5))
print("\nTarget feature in y:\n", y_train.head(5))

In [ ]:
# I will reset the indexes of the training and test splits so we can see the X_train printout
# We will see that they are no longer in order and the next markdown cell I will reset the indexes.
X_train.head(5)

In [ ]:
# Using .reset_index 
# We see that they are in order again. 
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
X_train.head(10)

***

<br><br>
# 2. Linear Regression

In this section, I will be preparating a linear regression model. I will attempt to see the prediction overall.

## 2.1 Training a linear regression model to predict the target feature.

In [ ]:
linear_reg = LinearRegression().fit(X_train, y_train)

## 2.2 Printing out the coefficients learned by the model and discussing the role in the model.

In [ ]:
print("\nThe features are: \n", X_train.columns)
print("\nThe coefficients are: \n", linear_reg.coef_)
print("\n The intercept is: \n", linear_reg.intercept_)
print("\nFeatures and coefficients: \n", list(zip(X_train.columns, linear_reg.coef_[0])))

## 2.3 Printing the predicted target feature. Printing the predicted class for a few examples. Printing classification evaluation measures computed on the full training set. 

In [ ]:
# Calculating the prediction and threshold value. 
linear_predictions_train_data = (linear_reg.predict(X_train))

In [ ]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([y_train, pd.DataFrame(linear_predictions_train_data, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg.head(50))

In [ ]:
# Printing the a few classification evaluation measures computed on the full training set.
# The following will be printed: accuracy, confusion matrix, precision, recall, f1).
# Some more evaluation metrics.
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_train, linear_predictions_train_data)))
print("MSE Score: ", metrics.mean_squared_error(y_train, linear_predictions_train_data))
print("MAE Score: ", metrics.mean_absolute_error(y_train, linear_predictions_train_data))
print("R2 Score: ", metrics.r2_score(y_train, linear_predictions_train_data))

## 2. 4 Evaluating the model using classification evaluation measures on the hold-out (30% examples) test set.

In [ ]:
linear_predictions_testing = (linear_reg.predict(X_test))

print("\nPredictions that has multiple linear regression: \n")
actual_vs_predicted_multiple_linear_reg = pd.concat([y_test, pd.DataFrame(linear_predictions_testing, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiple_linear_reg)

In [ ]:
# Printing the a few classification evaluation measures computed on the full training set.
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_test, linear_predictions_testing)))
print("MSE Score: ", metrics.mean_squared_error(y_test, linear_predictions_testing))
print("MAE Score: ", metrics.mean_absolute_error(y_test, linear_predictions_testing))
print("R2 Score: ", metrics.r2_score(y_test, linear_predictions_testing))

In [ ]:
# Taking a tripid from 46a and applying a prediction using Linear Regression model.
route_46a = df[(df['TRIPID'] == '8591174') & (df['DAYOFSERVICE']=='2018-12-23')]
route_46a = route_46a.drop(low_corr, 1)
route_46a = pd.get_dummies(route_46a)
actualtimes_46a = pd.DataFrame(route_46a['ACTUALTIME_TRAVEL'])
actualtimes_46a.reset_index(drop=True, inplace=True)
route_46a = route_46a.drop('ACTUALTIME_TRAVEL', 1)

In [ ]:
prediction_46a = linear_reg.predict(route_46a)
prediction_46a[0] = 0.0

In [ ]:
route_46a

In [ ]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted = pd.concat([actualtimes_46a, pd.DataFrame(prediction_46a, columns=['Prediction'])], axis=1, join='outer')
actual_vs_predicted.head(50)

In [ ]:
# Printing evaluation metrics
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(actualtimes_46a, prediction_46a)))
print("MSE Score: ", metrics.mean_squared_error(actualtimes_46a, prediction_46a))
print("MAE Score: ", metrics.mean_absolute_error(actualtimes_46a, prediction_46a))
print("R2 Score: ", metrics.r2_score(actualtimes_46a, prediction_46a))

***

<br><br>

# 3. Route model and taking the proportion of the prediction to calculate a journey time for the user.

## 3.1 Calculating the proportion of each stop from the overall trip.

In [ ]:
def proportion_stops(predictions):
    # Sum from the first stop until each stop
    sum_each_stop = np.zeros(predictions.shape[0], dtype=float)
    proportion_each_stop = np.zeros(predictions.shape[0], dtype=float)
    overall_prediction = np.sum(predictions)
    
    # Adding sum up until current stop and dividing by overall prediction to get proportion of the trip
    for length in range(predictions.shape[0]):
        sum_each_stop = np.append(sum_each_stop, [predictions[length]])
        sum_overall = np.sum(sum_each_stop) / overall_prediction*100
        proportion_each_stop[length] = sum_overall
        
    return proportion_each_stop

## 3.2 Return the progrnumber based off the stoppointid in a route

Finding the most common progrnumber based off the stoppointid. The reason for using to find the most common progrnumber is because it assumes that most route_id for each line would be always complete with the exception of a few trips in which they take a different route and skips some stops as a result.

In [ ]:
# Code taken from https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/

# array only accepts a panda Series or numpy array
def most_common(array):
    List = array.tolist()
    mode_list = mode(List)
    if mode_list == '1':
        return 0
    
    else:
        return(mode(List))

## 3.3 Calculating the journey time from a start to end destination based on user input

Finding the travel time duration based on a stoppointid then getting the progrnumber

In [ ]:
def journey_time(start,end, prediction):
    # Converting into int because the function returns a string
    start_progrnum = int(most_common(df['PROGRNUMBER'][df['STOPPOINTID']==start]))
    end_progrnum = int(most_common(df['PROGRNUMBER'][df['STOPPOINTID']==end]))
    
#     print(start_progrnum)
#     print(end_progrnum)

    proportion_array = proportion_stops(prediction)
    overall_prediction = np.sum(prediction)
    
    # calculating the time difference from start to end destination 
    start_prediction = (proportion_array[start_progrnum]/100) * overall_prediction
    end_prediction = (proportion_array[end_progrnum]/100) * overall_prediction
    
    journeytime = end_prediction - start_prediction
    
    # print(journeytime)
    
    return journeytime

In [ ]:
user_start = '807'
user_end = '812'

journey_time(user_start, user_end, prediction_46a)

***

<br><br>
# 4. Stop pair model

## 4.1 Pairing each stop based on the route - this approach has been disregarded

In [ ]:
# Storing one trip 
first_trip = df[df['TRIPID']=='5955251']
list_stopid = first_trip['STOPPOINTID'].tolist()

In [ ]:
def pairing_stops(stops_df):
    list_stopid = stops_df['STOPPOINTID'].tolist()
    
    # Creating pair stops for a route
    paired_stop = np.array(list(zip(list_stopid, list_stopid[1:] + list_stopid[:1])), dtype=int)

    # Removing last element from list (because it is a pair of the first and last stopid)
    paired_stop = paired_stop[:-1]
    
    return paired_stop

In [ ]:
def return_paired_stop(user_input, paired_stops):
    stops = []
    for i in range(len(paired_stops)):
        if paired_stops[i][1] == user_input:
#             print(user_input, paired_stops[i])
            stops += paired_stops[i]
    return stops

In [ ]:
def stops_df(input_stops):
    for first, second in pair_stop:
        print(first, second)
#         temp_df = df[df['STOPPOINTID'] == first]
#         temp_df = temp_df.append(df[df['STOPPOINTID']== second])

## 4.2 First version of paired stop approach
<br><br>
This approach makes a model based on the stopid and its previous stopids

In [ ]:
# Returns a paired list of stops
def paired_stops(df):
    stopid = df['STOPPOINTID'].unique().tolist()
    previous_stopid = []
    for i in stopid:
        prev = df['PREVIOUS_STOPPOINTID'][df['STOPPOINTID']==i]
        # Adds most frequent previous stopid to list
        previous_stopid += [prev.value_counts().idxmax()]
    
    return [stopid, previous_stopid]

In [ ]:
for ids in range(len(paired_stops[0])):
    
    # Making new dataframe
    to_add = df[df['STOPPOINTID']==paired_stops[0][ids]]
    to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==paired_stops[1][ids]])
    stops_df = pd.DataFrame(data=to_add)
    
    # Setting target feature
    y = stops_df['ACTUALTIME_TRAVEL']
    
    # Dropping target feature and low corr features
    stops_df = stops_df.drop(low_corr,1)
    stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
    stops_df = pd.get_dummies(stops_df)
    
    # Fitting model based on stops
    linear_reg = LinearRegression().fit(stops_df, y)
    
    # Save to pickle file 

In [ ]:
pair_stops = paired_stops(df)
to_add = df[df['STOPPOINTID']==pair_stops[0][5]]
to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==pair_stops[1][5]])
stops_df = pd.DataFrame(to_add)

 # Setting target feature
y = stops_df['ACTUALTIME_TRAVEL']
    
# Dropping target feature and low corr features
stops_df = stops_df.drop(low_corr,1)
stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
stops_df = pd.get_dummies(stops_df)

# Fitting/Training model based on stops
linear_reg_model_ = LinearRegression().fit(stops_df, y)

# Saving to pickle File
with open('model_'+pair_stops[0][5]+'.pkl', 'wb') as handle:
    pickle.dump(linear_reg_model_, handle)


In [ ]:
sampledf = stops_df.iloc[[0]]
sample_prediction = linear_reg_sample.predict(sampledf)

In [ ]:
sample_prediction

In [ ]:
with open('model_'+pair_stops[0][5]+'.pkl', 'rb') as handle:
    model = pickle.load(handle)

In [ ]:
model.predict(sampledf)

## 4.2.1 Setting up for 46a stop pair models using first approach

In [ ]:
# Function to get previous stopid and return a paired list
def pair_stopids(current_stopids):
    previous_stopid = []
    for i in current_stopids:
        prev = df['PREVIOUS_STOPPOINTID'][df['STOPPOINTID']==i]
        # Adds most frequent previous stopid to list
        previous_stopid += [prev.value_counts().idxmax()]
    
    return [current_stopids, previous_stopid]

In [ ]:
# Loading the json file
import json
file = open('routes_and_stops.json',)
routes_stops = json.load(file)

In [ ]:
# Get all stops for 46a going outbound ('1')
list_46a_stops = routes_stops['46A']['outbound']

# Pairing stopids and prev stopids from 46a route
pairing_46a_stopids = pair_stopids(list_46a_stops)
predictions = []

In [ ]:
for ids in range(len(pairing_46a_stopids[0])):
    # Making new dataframe
    to_add = df[df['STOPPOINTID']==pairing_46a_stopids[0][ids]]
    to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==pairing_46a_stopids[1][ids]])
    stops_df = pd.DataFrame(data=to_add)
    
    # Setting target feature
    y = stops_df['ACTUALTIME_TRAVEL']
    
    # Dropping target feature and low corr features
    stops_df = stops_df.drop(low_corr,1)
    stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
    stops_df = pd.get_dummies(stops_df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(stops_df, y)
    
      # Save to pickle file
#     with open('model_'+pairing_46a_stopids[0][ids]+'.pkl', 'wb') as handle:
#         pickle.dump(linear_reg_model, handle)

     # Predicting data
    with open('stop_'+pair_stops[0][ids]+'.pkl', 'rb') as handle:
        model = pickle.load(handle)
    
    k = model.predict(route_46a.iloc[[index]])
    predictions += [k]

In [ ]:
# Printing evaluation metrics
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(actualtimes_46a, predictions)))
print("MSE Score: ", metrics.mean_squared_error(actualtimes_46a, predictions))
print("MAE Score: ", metrics.mean_absolute_error(actualtimes_46a, predictions))
print("R2 Score: ", metrics.r2_score(actualtimes_46a, predictions))

<br><br>
##### Conclusion:
Linear regression model is not very good. MSE score is off by more than 1000 seconds. And the R2 score is at a negative value. This means the parameters need to be tuned. Keeping dwelltime might be good.

## 4.3 Stop pair based on entire leavetimes

In [ ]:
# Loading the json file
import json
file = open('routes_and_stops.json',)
routes_stops = json.load(file)

<br><br>
1) Make a rough query that selects rows that contain a certain stopid and its previous stopid based on the direction.

In [ ]:
# initialise query - for OUTBOUND (WHERE DIRECTION == '1')
query_stopid = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.STOPPOINTID = " + current_stopid + " AND leavetimes.DAYOFSERVICE = weather.dt"
query_stopid_df = pd.read_sql(query_previoustop, conn)

<br><br>
2) Make a function that will combine lists in a list together as one list

In [ ]:
def combine_listsoflist(to_combine):
    combined = []
    for each_list in to_combine:
        combined += each_list
    return combined

<br><br>
4) Make a function that will get rid of the duplicates in the list

In [ ]:
def get_unique(stopids_list):
    return list(set(stopids_list))

<br><br>
5) Make a list to store all stopids for DIRECTION == outbound/1.

In [ ]:
# Looping through every lineid, outbound 
stopids_outbound = []
for i,j in routes_stops.items():
    try:
#         print(i, '\n', routes_stops[i]['outbound'], '\n')
        stopids_outbound += [routes_stops[i]['outbound']]
    except KeyError:
        continue
        
# Calling function to get combined list
combined_stopids_outbound = combine_listsoflist(stopids_outbound)

# Calling function to get unique stopids from combined list
unique_stopids_outbound = get_unique(combined_stopids_outbound)

<br><br>
6) Make a list to store all stopids for DIRECTION ==inbound/2.

In [ ]:
# Looping through every lineid, inbound
stopids_inbound = []
for i,j in routes_stops.items():
    try:
#         print(i, '\n', routes_stops[i]['inbound'], '\n')
        stopids_inbound += [routes_stops[i]['inbound']]
    except KeyError:
        continue
        
# Calling function to get combined list
combined_stopids_inbound = combine_listsoflist(stopids_inbound)

# Calling function to get unique stopids from combined list - using set() to get rid off existing stops from outbound stops
unique_stopids_inbound = list(set(combined_stopids_inbound) - set(combined_stopids_outbound))

<br><br>
7) Query to select all of the previous_stopids based on the current stopid and put it to a list

In [ ]:
query_previoustop = "SELECT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + current_stopid 
query_prevstop_df = pd.read_sql(query_previoustop, conn)

# Converting into a pandas series then to list
query_prevstop_series = query_prevstop_df.iloc[0]
query_prevstop_list = query_prevstop_series.tolist()

<br><br>
8) Query to select the rows based on the previous stopids and append them to the current dataframe of the current stopid


In [ ]:
def df_prev_stops(query_prevstop_list):
    df_final = pd.DataFrame()
    # Loop through the list
#     for previous_stopid in query_prevstop_list:
    query_prevstop_rows = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.PREVIOUS_STOPPOINTID IN " + str(query_prevstop_list) + " AND leavetimes.DAYOFSERVICE = weather.dt"
    df_prevstop = pd.read_sql(query_prevstop_rows, conn)
    df_final = df_final.append(df_prevstop)
    return df_final

<br><br>
9) Adding index on STOPPOINTID and PREVIOUS_STOPPOINTID

In [ ]:
# Adding indexes
add_index1 = """CREATE INDEX stopid ON leavetimes(STOPPOINTID);"""
add_index2 = """CREATE INDEX previous_stopid ON leavetimes(PREVIOUS_STOPPOINTID);"""
conn.execute(add_index1)
conn.execute(add_index2)

<br><br>
10) Piecing every step together

In [ ]:
# import boto3
import pandas as pd
import numpy as np
import sqlite3
import pickle

# from sagemaker import get_execution_role
from sklearn.linear_model import LinearRegression
from math import log


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Connecting to s3
# role = get_execution_role()
# bucket='sagemaker-studio-520298385440-7in8n1t299'
# data_key = 'route_46a.feather'
# data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
low_corr = ['DAYOFSERVICE', 'VEHICLEID', 'TRIPID', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID', 'PROGRNUMBER', 'temp', 'pressure', 'humidity', 
            'wind_speed', 'wind_deg', 'weather_id', 'weather_description', 'clouds_all', 'PREVIOUS_STOPPOINTID', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP',
           'PLANNEDTIME_TRAVEL', 'DWELLTIME']

In [ ]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except 'Error' as e:
        print(e)
        
    return conn

In [ ]:
# create connection to db
db_file = "C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/dublinbus.db"
conn = create_connection(db_file)

In [ ]:
# Outbound
# test_df = pd.DataFrame()

for current_stopid in range(len(stopids_outbound[0])):
    # Making query to db and make df
    query_stopid = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.STOPPOINTID = " + stopids_outbound[0][current_stopid] + " AND leavetimes.DAYOFSERVICE = weather.dt"
    df = pd.read_sql(query_stopid, conn)

    # Get all previous stopids in list
    query_previoustop = "SELECT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + stopids_outbound[0][current_stopid] 
    query_prevstop_df = pd.read_sql(query_previoustop, conn)

    # Converting into a pandas series then to list
    query_prevstop_series = query_prevstop_df['PREVIOUS_STOPPOINTID'].tolist()
#     print(query_prevstop_series)
#     query_prevstop_list = query_prevstop_series.tolist()
#     print(query_prevstop_list)
    query_prevstop_list = [stopid for stopid in query_prevstop_series if stopid != '0']
    query_prevstop_list = tuple(get_unique(query_prevstop_list))
    
    # Append previous stops rows to main df
    to_add = df_prev_stops(query_prevstop_list)
    df = df.append(to_add)
    
    print('Concatentaion finished')
    
    # Drop low correlated features and setting target feature
    df = df.drop(low_corr, 1)
    tf = df['ACTUALTIME_TRAVEL']
    df = df.drop('ACTUALTIME_TRAVEL', 1)
    df = pd.get_dummies(df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(df, tf)
    
    # Save to pickle file
    with open('/UCD/ResearchPracticum/Data-Analytics-CityRoute/stop_models/stop_'+ stopids_outbound[0][current_stopid] +'.pkl', 'wb') as handle:
        pickle.dump(linear_reg_model, handle)

In [ ]:
# Inbound 
for current_stopid in range(len(stopids_inbound)):
    # Making query to df and make df
    query_stopid = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.STOPPOINTID = " + stopids_inbound[current_stopid] + " AND leavetimes.DAYOFSERVICE = weather.dt"
    df = pd.read_sql(query_previoustop, conn)
    
    # Get all previous stopids in list
    query_previoustop = "SELECT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + stopids_inbound[current_stopid] 
    query_prevstop_df = pd.read_sql(query_previoustop, conn)

    # Converting into a pandas series then to list
    query_prevstop_series = query_prevstop_df.iloc[0]
    query_prevstop_list = query_prevstop_series.tolist()
    
    # Append previous stops rows to main df
    to_add = df_prev_stops(query_prevstop_list)
    df = df.append(to_add)
    
    # Drop low correlated features and setting target feature
    df = df.drop(low_corr, 1)
    tf = df['ACTUALTIME_TRAVEL']
    df = df.drop('ACTUALTIME_TRAVEL', 1)
    df = pd.get_dummies(df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(df, tf)
    
    # Save to pickle file
    with open('/UCD/ResearchPracticum/Data-Analytics-CityRoute/stop_models/stop_'+ stopids_outbound[current_stopid] +'.pkl', 'wb') as handle:
        pickle.dump(linear_reg_model, handle)

***

<br><br>
# 5. Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfm = RandomForestRegressor(n_estimators=40, oob_score=True, random_state=1)
rfm.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtc_4 = DecisionTreeRegressor(max_depth=4, random_state=1)
dtc_4.fit(X_train, y_train)

In [ ]:
importance = pd.DataFrame({'feature': X_train.columns, 'importance': rfm.feature_importances_})
importance.sort_values('importance', ascending=False)

In [ ]:
# Printing the evaluated mode using the hold-out training set
rfm_predictions_train = rfm.predict(X_train)
predicted_death_yn_Yes = pd.DataFrame(rfm_predictions_train, columns=['Predicted'])
actual_vs_predicted_rfm_train = pd.concat([y_train, predicted_death_yn_Yes], axis=1)
actual_vs_predicted_rfm_train.head(20)

In [ ]:
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_train, rfm_predictions_train)))
print("MSE Score: ", metrics.mean_squared_error(y_train, rfm_predictions_train))
print("MAE Score: ", metrics.mean_absolute_error(y_train, rfm_predictions_train))
print("R2 Score: ", metrics.r2_score(y_train, rfm_predictions_train))

In [ ]:
# Printing the evaluated mode using the hold-out training set
rfm_predictions_test = rfm.predict(X_test)
predicted_death_yn_test = pd.DataFrame(rfm_predictions_test, columns=['Predicted'])
actual_vs_predicted_rfm_test = pd.concat([y_test, predicted_death_yn_test], axis=1)
actual_vs_predicted_rfm_test.head(20)

In [ ]:
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_test, rfm_predictions_test)))
print("MSE Score: ", metrics.mean_squared_error(y_test, rfm_predictions_test))
print("MAE Score: ", metrics.mean_absolute_error(y_test, rfm_predictions_test))
print("R2 Score: ", metrics.r2_score(y_test, rfm_predictions_test))

In [ ]:
import datetime
c = int(df['ACTUALTIME_DEP'][500])

In [ ]:
import time
res = int(time.strftime("%H",time.gmtime(c)))
print(res >= 7 and res <= 9)